In [98]:
import requests
from bs4 import BeautifulSoup
import pymongo
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
%matplotlib inline
import re

In [141]:
import json
f = open("movie.json", "r")
movie_data = json.load(f)
f.close()
movie_data = movie_data
print(movie_data[0].keys())
line = ""
for i in movie_data[0:100]:
    line += "{}\t".format(i['Mname'])
print(line)

[u'realease_kor', u'Webpage', u'Mname', u'Domestic', u'budget', u'gross_kor', u'release_usa', u'year', u'runtime', u'Oversea', u'rating_usa']
Toy Story 3	Alice in Wonderland (2010)	Harry Potter and the Deathly Hallows Part 1	Inception	Shrek Forever After	The Twilight Saga: Eclipse	Iron Man 2	Tangled	Despicable Me	How to Train Your Dragon	Clash of the Titans (2010)	The Chronicles of Narnia: The Voyage of the Dawn Treader	The King's Speech	Tron Legacy	The Karate Kid	Prince of Persia: The Sands of Time	Black Swan	Megamind	Robin Hood	The Last Airbender	Little Fockers	Resident Evil: Afterlife	Shutter Island	Salt	Sex and the City 2	The Tourist	The Expendables	Knight & Day	True Grit	Gulliver's Travels	Percy Jackson & The Olympians: The Lightning Thief	The Social Network	Valentine's Day	The Sorcerer's Apprentice	Due Date	Eat Pray Love	Red	Paranormal Activity 2	The A-Team	Jackass 3-D	Unstoppable	Step Up 3-D	The Book of Eli	The Town	Legend of the Guardians: The Owls of Ga'Hoole	The Wolfman	The B

In [170]:
class Database(object):
    def __init__(self):
        self.connection = pymongo.MongoClient("localhost", 27017)
        self.db = self.connection["DSS4_SmallP"]
        self.collection = self.db["Movie_mojo"]        
def Load_Data_mojo_yearly_dom(page,year):
    Mojo_url = u'http://www.boxofficemojo.com/'\
        u'yearly/chart/?page={}&view=releasedate&view2=domestic&yr={}&p=.htm'.format(page,year)
    return requests.get(Mojo_url)
def Load_Data_mojo_yearly_intl(year):
    Mojo_url = u'http://www.boxofficemojo.com/'\
        u'yearly/chart/?view2=worldwide&yr={}&p=.htm'.format(year)
    return requests.get(Mojo_url)
def Load_Data_mojo_movie_intl(M_id):
    Mojo_url = u'http://www.boxofficemojo.com{}&page=intl'.format(M_id)
    return requests.get(Mojo_url)
def Load_Data_mojo_movie(M_id):
    Mojo_url = u'http://www.boxofficemojo.com{}'.format(M_id)
    return requests.get(Mojo_url)

#Daum crawler
def load_apikeys():
    fp = open("private.json","r")
    res = json.load(fp)
    return res
def load_data_daum(query):
    key = load_apikeys()['daum_key']
    try :
        url = "https://apis.daum.net/contents/movie?apikey={}&q={}&output=json".format(key, query)
    except UnicodeEncodeError :
        query2 = re.match('[a-zA-Z\s]+',query)
        url = "https://apis.daum.net/contents/movie?apikey={}&q={}&output=json".format(key, query2.group())
    except Exception as err:
        print(err)
#    print(url)
    return requests.get(url)
def select_daum(query, movie):
    res = load_data_daum(query)
#    print(res)
    data = json.loads(res.content)
    #검색결과가 0개일때, 검색하는 제목에서 영어랑 띄어쓰기만 필터링해서 (ex. Godzilla (2014) -> Godzilla) 재검색
    if data['channel']['totalCount'] == 0:
        try : 
            query2 = re.match('[a-zA-Z\s]+',query)
            if query2.group() == query:
                return -2
            else:
                return select_daum(query2.group(), movie)
        except AttributeError:
            return -2
        except Exception as err:
            print(err)
            return -2
    #검색결과가 1개일때 결과값 출력
    elif data['channel']['totalCount'] == 1:
        return data['channel']["item"][0]
    #검색결과가 여러개일때, 개봉일을 비교하여 같은경우 출력, 개봉일이 같은게 없는경우 출력안함.
    else :
        mlist = []
        try:
            m = movie['realease_kor'].split('/')
            m = '{0:02d}{1:02d}{2:02d}'.format(int(m[2]),int(m[0]),int(m[1]))
            for item in data['channel']['item']:
                try:
                    d = item['open_info'][0]['content'].split('.')
                    d = '{0:02d}{1:02d}{2:02d}'.format(int(d[0][-2:]),int(d[1]),int(d[2]))
                    if d == m :
                        mlist.append(item)
                except:
                    pass
            if len(mlist) == 1:
                return mlist[0]
            else :
                if len(mlist) != 0 :print(len(mlist))
                return -1
        except Exception as err:
            print(err)
            return -1

In [101]:
print(movie_data[0].keys())

[u'realease_kor', u'Webpage', u'Mname', u'Domestic', u'budget', u'gross_kor', u'release_usa', u'year', u'runtime', u'Oversea', u'rating_usa']


In [172]:
arr_movie = []
err_movie = []
for arr_num, movie in enumerate(movie_data[260:]):
    res = select_daum(movie['Mname'],movie)
    dict_movie = {
        'm_name_k' : 'NA',
        'grade' : 'NA',
        'viewing_time' : -1,
        'genre' : 'NA',
        'director_k' : 'NA',
        'actor_k' : 'NA'
    }
    if res == -1 :
        err = "multiple : {}".format(movie['Mname'])
        err_movie.append(err)
    elif res == -2 : 
        err = "none : {}".format(res)
        err_movie.append(err)
    else :
        if res['open_info'][2]['content'] == '' :  res['open_info'][2]['content'] = '-1 '
        dict_movie = {
            'm_name_k' : res['title'][0]['content'],
            'grade' : res['open_info'][1]['content'],
            'viewing_time' : int(res['open_info'][2]['content'][0:-1]),
            'genre' : [X["content"] for X in res["genre"]],
            'director_k' : [(X["content"], X["link"]) for X in res["director"]],
            'actor_k' : [(X["content"], X["link"]) for X in res["actor"]]
        }
    movie_data[arr_num].update(dict_movie)
fp = open("movie_added_daum.json", "w")
json.dump(movie_data, fp, indent = 4)
fp.close()

list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range


In [173]:
#몽고DB
db = Database()
for movie in movie_data:
    db.collection.update_one({'Mname' : movie['Mname']}, {"$set":movie}, upsert=True)